<font color='gray'>*Content provided under a Creative Commons Attribution license, CC-BY 4.0; code under a BSD-3-Clause License  
@ 2020 GitPistachio*
</font>

<h1><center>Liczby złożone z trzech różnych czynników pierwszych</center></h1>

<h2> Problem </h2>

<strong>Napisz program, który do pliku wyniki.txt wypisuje te liczby z pliku liczby.txt, w których rozkładzie na czynniki pierwsze występują dokładnie trzy różne czynniki pierwsze różne od 2. W pliku liczby.txt znajduje się 1000 różnych liczb o długości od 2 do 9 cyfr, po jednej w każdym wierszu.</strong>

Zanim przejdę do rozwiązania problemu. Zacznę od przypomnienia kilku podstawowych definicji. <em>Liczbą pierwszą </em> nazywamy dowloną liczbę naturalną większą od 1, która ma dokładnie dwa dzielniki: samą siebie i jedynkę. Kilka początkowych liczb pierwszych to 2, 3, 5, 7, 11, 13 itd. (patrz ciąg [A000040](https://oeis.org/A000040)). Każdą liczbę naturalną większą od 1 niebędącą liczbą pierwszą nazywamy <em>liczbą złożoną</em>. Zaś rozkład liczby naturalnej większej od 1 na czynniki pierwsze polega na jej zapisaniu jako iloczynu liczb pierwszych, np. $36 = 2^2\cdot3^2$. Program będzie składał się z trzech części: wczytania danych, sprawdzenia liczba po liczbie czy spełnia warunki zadania i następnie zapisania jej do pliku wyjściowego. Sprawdzenie czy w rozkładzie liczby występują dokładnie trzy czynniki różne do dwa będę robił w funkcji ```czySpelniaWarunekZadania```, która zwraca wartość ```True``` jeżeli spełnia warunek zadania a ```False``` jeżeli nie. Na razie założę, że tę funkcję po prostu mam, później zajmę się napisaniem jej treści. W treści zadania nie jest jasno napisane czy w rozkładzie liczby na czynniki pierwsze czynnik pierwszy może występować wielokrotnie, wobec tego dodam drugi parametr do funkcji o nazwie czy_wielokrotnosci, który będzie przyjmować wartości True lub False. Poniżej przedstawiam przykładowy program.

In [1]:
def czySpelniaWarunekZadania(n, czy_wielokrotnosci):
    pass # do uzupełnienia później


plik_liczby = open("data/liczby.txt", "r") # otwiera plik liczby w trybie do odczytu
plik_wyniki = open("data/wyniki.txt", "w") # otwiera plik wyniki w trybie do zapisu

liczby = plik_liczby.readlines() # odczytuje liczby z pliku i zapsuje je do listy jako zmienne tekstowe
for liczba in liczby:
    n = int(liczba) # konwertuje wartość liczba na zmienną typu int
    if czySpelniaWarunekZadania(n, True):
        plik_wyniki.write("{}\n".format(n)) # zapisuje liczbę do pliku wyniki oraz dodaje nową linię

plik_wyniki.close() # zamyka plik wyniki
plik_liczby.close() # zamyka plik liczby

 Alogrytm sprawdzający warunek zadania będzie następujący:
- sprawdzamy czy n dzieli się przez 2, jeżeli tak zwracamy False, jeżeli nie inicjalizujemy zerem licznik czynników pierwszych,
- sprawdzamy dla kolejnej liczby pierwszej p czy n jest przez nią podzielne, jeżeli tak oraz
    - licznik czynników pierwszych wynosi co najmniej trzy, zwróć False,
    - parametetr czy_wielokrotne = False i n jest podzielne przez kwadrat p, zwróć False,
    - zwiększ licznik czynników pierwszych o 1,
- powtórz krok drugi, dopuki p\*p <= n
- jeżeli licznik czynników pierwszych wskazuje na trzy zwróć True w przeciwnym przypadku zwróć False.

Sam algorytm jest dość prosty, drobnym problemem jest krok drugi, gdzie musimy znać kolejne liczby pierwsze. Do tego celu posłużę się jednym z najprostrzych i zarazem dość efektywnym algorytmem do generowania liczb pierwszych, którym jest [sito Eratostenesa](https://pl.wikipedia.org/wiki/Sito_Eratostenesa). Poniżej podaję podstawową implementację sita Eratostenesa.

In [2]:
from math import sqrt

def wygeneruj_liczby_pierwsze(n):
    """Generuję posortowaną listę liczb pierwszych, nie większych niż maksymalny dzielnik liczby n.

    Args:
        n (int): Liczba naturalna większa od 1.

    Returns:
        list: posortowana lista liczb pierwszych, nie większych niż maksymalny dzielnik liczby n.
    
    """
    
    P = [] # lista liczb pierwszych 
    max_p = int(sqrt(n)) # maksymalna potencjalna liczba pierwsza jaka może być dzielnikiem n
    A = [True for _ in range(max_p + 1)] # lista typów logicznych
    A[0], A[1] = False, False # 0, 1 nie są liczbami pierwszymi

    for i in range(2, max_p + 1):
        if A[i]:
            P.append(i)
            j = 2*i
            while j <= max_p: # oznacza wszystkie wielokrotności jako liczby złożone
                A[j] = False 
                j += i
    
    return P  
        
print(wygeneruj_liczby_pierwsze(100))

[2, 3, 5, 7]


W zadaniu jest podane, że n mam maksymalnie 9 cyfr wobec tego maksymalną liczbą w pliku jest 999999999 i dla tej liczby wygeneruję listę liczb pierwszych, gdyż będzie zawierała wszystkie możliwe liczby pierwsze, które moga być dzielnikami liczb z pliku liczby. Poniżej podaję imlementację funkcji ```czySpelniaWarunekZadania```.

In [3]:
MAX_N = 999999999 # maksymalna wartość n
P = wygeneruj_liczby_pierwsze(MAX_N)

def czySpelniaWarunekZadania(n, czy_wielokrotnosci):
    """Sprawdza czy w rozkładzie na czynniki pierwsze liczby n występuja dokładnie 
       trzy różne czynniki pierwsze różne od 2.

    Args:
        n (int): Liczba naturalna większa od 1.
        czy_wielokrotnosci (bool): True jeżeli mogą wysąpić wielokrotności dane czynnika pierwszego, False
                                   w przeciwnym przypadku

    Returns:
        bool: True jeżeli w rozkładzie na czynniki pierwsze liczby n wstępują dokładnie trzy
              różne czynniki pierwsze różne od 2, False w przeciwnym przypadku.
    
    """
    global P

    if n % 2 == 0: # sprawdza czy n dzieli się przez 2 za pomocą dzielenia modulo
        return False # zwraca False gdyż liczba n nie może mieć 2 jako czynnik pierwszy

    licznik_czynnikow_pierwszych = 0
    
    for p in P[1:]:
        if p*p > n:
            break
        
        if n % p == 0: # sprawdza czy n jest podzielne przez p
            if licznik_czynnikow_pierwszych >= 3:
                return False
            
            if czy_wielokrotnosci == False and (n % (p*p)) == 0:
                return False
            
            licznik_czynnikow_pierwszych += 1
            
    
    if licznik_czynnikow_pierwszych == 3:
        return True
    else:
        return False

print("Czy {} spełnia warunek zadania? {}".format(36, czySpelniaWarunekZadania(36, True)))
print("Czy {} spełnia warunek zadania? {}".format(825, czySpelniaWarunekZadania(825, True)))
print("Czy {} spełnia warunek zadania, gdy czynniki nie mogą być wielokrotne? {}".format(825, czySpelniaWarunekZadania(825, False)))

Czy 36 spełnia warunek zadania? False
Czy 825 spełnia warunek zadania? True
Czy 825 spełnia warunek zadania, gdy czynniki nie mogą być wielokrotne? False


Poniżej przedstawiam jeszcze cały kod łącznie.

In [4]:
from math import sqrt

def wygeneruj_liczby_pierwsze(n):
    """Generuję posortowaną listę liczb pierwszych, nie większych niż maksymalny dzielnik liczby n.

    Args:
        n (int): Liczba naturalna większa od 1.

    Returns:
        list: posortowana lista liczb pierwszych, nie większych niż maksymalny dzielnik liczby n.
    
    """
    
    P = [] # lista liczb pierwszych 
    max_p = int(sqrt(n)) # maksymalna potencjalna liczba pierwsza jaka może być dzielnikiem n
    A = [True for _ in range(max_p + 1)] # lista typów logicznych
    A[0], A[1] = False, False # 0, 1 nie są liczbami pierwszymi

    for i in range(2, max_p + 1):
        if A[i]:
            P.append(i)
            j = 2*i
            while j <= max_p: # oznacza wszystkie wielokrotności jako liczby złożone
                A[j] = False 
                j += i
    
    return P  
        

MAX_N = 999999999 # maksymalna wartość n
P = wygeneruj_liczby_pierwsze(MAX_N)

def czySpelniaWarunekZadania(n, czy_wielokrotnosci):
    """Sprawdza czy w rozkładzie na czynniki pierwsze liczby n występuja dokładnie 
       trzy różne czynniki pierwsze różne od 2.

    Args:
        n (int): Liczba naturalna większa od 1.
        czy_wielokrotnosci (bool): True jeżeli mogą wysąpić wielokrotności dane czynnika pierwszego, False
                                   w przeciwnym przypadku

    Returns:
        bool: True jeżeli w rozkładzie na czynniki pierwsze liczby n wstępują dokładnie trzy
              różne czynniki pierwsze różne od 2, False w przeciwnym przypadku.
    
    """
    global P

    if n % 2 == 0: # sprawdza czy n dzieli się przez 2 za pomocą dzielenia modulo
        return False # zwraca False gdyż liczba n nie może mieć 2 jako czynnik pierwszy

    licznik_czynnikow_pierwszych = 0
    
    for p in P[1:]:
        if p*p > n:
            break
        
        if n % p == 0: # sprawdza czy n jest podzielne przez p
            if licznik_czynnikow_pierwszych >= 3:
                return False
            
            if czy_wielokrotnosci == False and (n % (p*p)) == 0:
                return False
            
            licznik_czynnikow_pierwszych += 1
            
    
    if licznik_czynnikow_pierwszych == 3:
        return True
    else:
        return False


plik_liczby = open("data/liczby.txt", "r") # otwiera plik liczby w trybie do odczytu
plik_wyniki = open("data/wyniki.txt", "w") # otwiera plik wyniki w trybie do zapisu

liczby = plik_liczby.readlines() # odczytuje liczby z pliku i zapsuje je do listy jako zmienne tekstowe
for liczba in liczby:
    n = int(liczba) # konwertuje wartość liczba na zmienną typu int
    if czySpelniaWarunekZadania(n, True):
        plik_wyniki.write("{}\n".format(n)) # zapisuje liczbę do pliku wyniki oraz dodaje nową linię

plik_wyniki.close() # zamyka plik wyniki
plik_liczby.close() # zamyka plik liczby